In [41]:
import pdfplumber
import pandas as pd
import os
import re
from collections import defaultdict
from unicodedata import normalize

In [ ]:
for file in os.listdir("./from"):
    print(f"current file: {file}")
    name_from = "./from/" + file
    name_to = "./to/" + file.replace(".pdf", ".txt")

    pdf_file = pdfplumber.open(name_from)
    w_file = open(name_to, "w", encoding="utf-8")

    # ① ~ ⑤ -> chr(9312) ~ chr(9316)
    # 한글 -> chr(44032 ~ 55203)
    # 한국용 한자 -> chr(19968 ~ 40959)

    for page in pdf_file.pages:
        print(f"{page} :", end=" ")
        left, right = [], []

        for word in page.extract_words():
            if word["x0"] < 400:
                target = left
            else:
                target = right

            if (re.match(r"\d{1,2}\.", word["text"]) # n.
                or re.match(r"\[\d{1,2}～\d{1,2}\]", word["text"]) # [m～n]
                or 9312 <= ord(word["text"][0]) <= 9316):  # ① ~ ⑤
                target.append("\n")

            target.append(word["text"])

        data = pd.DataFrame([*left, *right])

        if len(data):
            print("w /", end=" ")
            w_file.write(" ".join(data[0].tolist())+ "\n")
        else:
            print("n /", end=" ")

    pdf_file.close()
    w_file.close()
    print("")
print("done.")

In [42]:
f = open("./hanja_translated/2023학년도-6월-모의평가-경제-문제.txt", "r", encoding="utf-8")

hanjas = defaultdict(int)

for letter in f.read():
    if 19968 <= ord(normalize("NFKC", letter)) <= 29973:
        hanjas[letter] += 1
f.close()
data = pd.DataFrame([[k for k in hanjas.keys()], [v for v in hanjas.values()]]).T
data.sort_values(by=1, ascending=False)

,0,1
76,國,70
116,材,33
59,個,31
52,價,31
114,年,31
...,...,...
142,慮,1
133,員,1
132,新,1
129,切,1


In [38]:
data.loc[:, 1].apply(lambda x: x ** 0.1,).sort_values(ascending=False)

76     1.529360
116    1.418572
52     1.409731
59     1.409731
114    1.409731
         ...   
214    1.000000
217    1.000000
216    1.000000
220    1.000000
221    1.000000
Name: 1, Length: 222, dtype: float64